In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [4]:
# create engine to hawaii.sqlite
database_path="Resources/hawaii.sqlite"
engine = create_engine(f"sqlite:///{database_path}")

In [5]:
# reflect an existing database into a new model
Base=automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

In [6]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [7]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [8]:
# Create our session (link) from Python to the DB
session = Session(engine)


# Exploratory Precipitation Analysis

In [11]:
# Find the most recent date in the data set.
last_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
last_date

('2017-08-23',)

In [12]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
precipitation = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date >= last_date).\
    order_by(Measurement.date).all()
precipitation



InterfaceError: (sqlite3.InterfaceError) Error binding parameter 0 - probably unsupported type.
[SQL: SELECT measurement.date AS measurement_date, measurement.prcp AS measurement_prcp 
FROM measurement 
WHERE measurement.date >= ? ORDER BY measurement.date]
[parameters: (('2017-08-23',),)]
(Background on this error at: http://sqlalche.me/e/14/rvf5)

In [ ]:
# Calculate the date one year from the last date in data set.
last_date = dt.date(2017,8,23) - dt.timedelta(days=365)
last_date

In [ ]:
# Perform a query to retrieve the data and precipitation scores
scores = session.query(Measurement.date, Measurement.prcp).order_by(Measurement.date.desc()).all()
scores

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
precipitation_df = pd.DataFrame(precipitation,columns=["Date","Precipitation"]).set_index("Date")
precipitation_df

In [ ]:
# Sort the dataframe by date
precipitation_df.sort_values(by="Date",ascending=True)

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
precipitation_df.plot(title="Precipitation Analysis", figsize=(12,7))
plt.legend(loc='upper center')
plt.savefig("images/precip.png")
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
precipitation_df.describe()


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
stations_count = session.query(Measurement.station).distinct().count()
stations_count

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
most_active_stations = session.query(Measurement.station, func.count(Measurement.station)).\
    group_by(Measurement.station).\
    order_by(func.count(Measurement.station).desc()).all()
most_active_stations

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
sel = [func.min(Measurement.tobs),func.max(Measurement.tobs), func.avg(Measurement.tobs)]
temperatures = session.query(*sel).filter(Measurement.station==most_active_stations[0][0]).all()
print(f"The lowest temperature of {most_active_stations[0][1]} station ({most_active_stations[0][0]}) is: {temperatures[0][0]}°F")
print(f"The highest temperature of {most_active_stations[0][1]} station ({most_active_stations[0][0]})) is: {temperatures[0][1]}°F")
print(f"The average temperature of {most_active_stations[0][1]} station ({most_active_stations[0][0]})) is: {temperatures[0][1]}°F")

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
station_id = [func.min(Measurement.tobs), 
           func.max(Measurement.tobs), 
           func.avg(Measurement.tobs)]

min_max_avg_temp = session.query(*station_id).\
    filter(Measurement.station == "USC00519281").all()
min_max_avg_temp

# Close session

In [ ]:
# Close Session
session.close()